In [1]:
import random
import multiprocessing
from multiprocessing import Process, Lock
import time
import os

Генерация 50000 случайных 32-битных целых чисел.

In [2]:
numbers = [random.randint(1, 2**32 - 1) for _ in range(50000)]

In [3]:
def count_prime_factors(x):
    cnt, i = 0, 2
    while i * i <= x:
        while x % i == 0:
            cnt +=1
            x /= i
        i += 1
    if x > 1:
        cnt += 1
    return cnt

Подсчет простым последовательным алгоритмом


In [4]:
def single_thread_counter(numbers):
    total_cnt = 0
    for x in numbers:
        total_cnt += count_prime_factors(x)
    return total_cnt

In [6]:
start_time = time.time()
result = single_thread_counter(numbers)
end_time = time.time()
print(f"Результат: {result}")
print(f"Время выполнения: {end_time - start_time} секунд")

Результат: 206411
Время выполнения: 85.0718731880188 секунд


Подсчет многопоточно

In [13]:
def thread_counter(total_cnt, now, lock, numbers):
    res, i = 0, 0
    with lock:
        i = now.value
        now.value += 1
    while i < len(numbers):
        res += count_prime_factors(numbers[i])
        with lock:
            i = now.value
            now.value += 1
    with lock:
        total_cnt.value += res

In [14]:
def multiprocessing_counter(numbers):
    total_cnt, now = multiprocessing.Value('i', 0), multiprocessing.Value('i', 0)
    lock = multiprocessing.Lock()
    processes = []
    for i in range(os.cpu_count()):
        process = multiprocessing.Process(target=thread_counter, args=(total_cnt, now, lock, numbers))
        processes.append(process)
        process.start()
    for process in processes:
        process.join()
    return total_cnt.value

In [15]:
start_time = time.time()
result = multiprocessing_counter(numbers)
end_time = time.time()
print(f"Результат: {result}")
print(f"Время выполнения: {end_time - start_time} секунд")

Результат: 206411
Время выполнения: 88.496661901474 секунд


Подсчет с помощью Ray

In [11]:
!pip install ray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 MB 13.5 MB/s eta 0:00:00


In [12]:
import ray
import time

ray.init()

@ray.remote
def prime_factors_count(number):
    return count_prime_factors(number)


start_time = time.time()
results = ray.get([prime_factors_count.remote(number) for number in numbers])

total_prime_factors = sum(results)

end_time = time.time()

ray.shutdown()

print("Алгоритм с использованием Ray:")
print(f"Общее количество простых множителей: {total_prime_factors}")
print(f"Время выполнения: {end_time - start_time} секунд")

2024-07-03 10:06:12,506	INFO worker.py:1771 -- Started a local Ray instance.


Алгоритм с использованием Ray:
Общее количество простых множителей: 206411
Время выполнения: 156.35539412498474 секунд
